In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
# Завантаження даних 
data = pd.read_csv(r'cheese_data.csv')

In [3]:
# Виведення колонок
data.columns

Index(['CheeseId', 'ManufacturerProvCode', 'ManufacturingTypeEn',
       'MoisturePercent', 'FlavourEn', 'CharacteristicsEn', 'Organic',
       'CategoryTypeEn', 'MilkTypeEn', 'MilkTreatmentTypeEn', 'RindTypeEn',
       'CheeseName', 'FatLevel'],
      dtype='object')

In [4]:
# Заповнення числових NaN медіаною
for column in data.select_dtypes(include=[np.number]).columns:
    data[column].fillna(data[column].median(), inplace=True)

C:\Users\maksy\AppData\Local\Temp\ipykernel_10960\3279476893.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].median(), inplace=True)
C:\Users\maksy\AppData\Local\Temp\ipykernel_10960\3279476893.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [5]:
# Встановлюємо CategoryTypeEn як таргет
target = 'CategoryTypeEn'
data[target] = LabelEncoder().fit_transform(data[target])

In [6]:
# Підготовка даних
data_encoded = pd.get_dummies(data.drop(columns=[target, 'CheeseId', 'CheeseName']), drop_first=True)
X = data_encoded
y = data[target]

In [7]:
# Нормалізація
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
# Розділення на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
X_train[0]

array([-0.53079542,  3.08630078, -0.25793448, ...,  1.2566654 ,
       -0.40687602,  0.72345823])

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

# Перетворення даних у тензори
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Створення DataLoader для тренувальних і тестових даних
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Модель повнозв'язаної нейронної мережі
class FullyConnectedNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FullyConnectedNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

# Ініціалізація моделі, оптимізатора і функції втрат
input_dim = X_train.shape[1]
num_classes = len(set(y_train))
model = FullyConnectedNN(input_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Тренування моделі
epochs = 20
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Оцінка моделі
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(y_batch.numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Epoch 1/20, Loss: 43.4324, Accuracy: 22.63%
Epoch 2/20, Loss: 35.1455, Accuracy: 38.96%
Epoch 3/20, Loss: 29.4354, Accuracy: 58.30%
Epoch 4/20, Loss: 22.1171, Accuracy: 72.02%
Epoch 5/20, Loss: 15.2687, Accuracy: 82.17%
Epoch 6/20, Loss: 11.1140, Accuracy: 85.46%
Epoch 7/20, Loss: 9.2344, Accuracy: 86.83%
Epoch 8/20, Loss: 7.9497, Accuracy: 89.16%
Epoch 9/20, Loss: 6.9756, Accuracy: 90.67%
Epoch 10/20, Loss: 6.7650, Accuracy: 90.81%
Epoch 11/20, Loss: 6.3836, Accuracy: 91.08%
Epoch 12/20, Loss: 6.0302, Accuracy: 91.77%
Epoch 13/20, Loss: 5.3447, Accuracy: 92.59%
Epoch 14/20, Loss: 5.3849, Accuracy: 91.91%
Epoch 15/20, Loss: 4.8863, Accuracy: 93.00%
Epoch 16/20, Loss: 4.8556, Accuracy: 93.14%
Epoch 17/20, Loss: 4.8212, Accuracy: 93.14%
Epoch 18/20, Loss: 4.7545, Accuracy: 92.87%
Epoch 19/20, Loss: 4.9367, Accuracy: 93.14%
Epoch 20/20, Loss: 4.2364, Accuracy: 94.10%

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.61      0.60 

C:\Users\maksy\anaconda3\envs\test_rag\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\maksy\anaconda3\envs\test_rag\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\maksy\anaconda3\envs\test_rag\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.metrics import classification_report

# Гіперпараметри
batch_size = 64
learning_rate = 0.001
epochs = 30

# Завантаження та трансформація датасету STL10
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Завантаження повного датасету
full_train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
full_test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Вибір підмножини з 1000 об'єктів
np.random.seed(42)  # Для відтворюваності
train_indices = np.random.choice(len(full_train_dataset), 800, replace=False)  # 800 об'єктів для тренування
test_indices = np.random.choice(len(full_test_dataset), 200, replace=False)  # 200 об'єктів для тестування

train_dataset = Subset(full_train_dataset, train_indices)
test_dataset = Subset(full_test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Огляд класів
classes = full_train_dataset.classes
print("Classes:", classes)

# Згорткова нейронна мережа
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # RGB (3 канали)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)  # 10 класів
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Ініціалізація моделі, оптимізатора і функції втрат
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Тренування
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Оцінка
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))


Files already downloaded and verified
Files already downloaded and verified
Classes: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
Epoch 1/30, Loss: 29.8972, Accuracy: 11.12%
Epoch 2/30, Loss: 29.1299, Accuracy: 15.88%
Epoch 3/30, Loss: 27.9596, Accuracy: 19.38%
Epoch 4/30, Loss: 26.2478, Accuracy: 24.38%
Epoch 5/30, Loss: 24.7641, Accuracy: 28.88%
Epoch 6/30, Loss: 23.4825, Accuracy: 32.75%
Epoch 7/30, Loss: 22.3756, Accuracy: 36.25%
Epoch 8/30, Loss: 20.8989, Accuracy: 39.38%
Epoch 9/30, Loss: 20.8231, Accuracy: 39.75%
Epoch 10/30, Loss: 19.3130, Accuracy: 45.00%
Epoch 11/30, Loss: 18.5182, Accuracy: 48.88%
Epoch 12/30, Loss: 17.5198, Accuracy: 51.25%
Epoch 13/30, Loss: 15.7409, Accuracy: 57.25%
Epoch 14/30, Loss: 14.1029, Accuracy: 59.38%
Epoch 15/30, Loss: 13.6625, Accuracy: 63.12%
Epoch 16/30, Loss: 11.7604, Accuracy: 67.38%
Epoch 17/30, Loss: 11.3432, Accuracy: 69.38%
Epoch 18/30, Loss: 10.6285, Accuracy: 70.00%
Epoch 19/30, Loss: 8.4119, A

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords

# Завантаження стоп-слів
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Завантаження даних
data = pd.read_csv('twitter_training.csv')
data.columns = ['ID', 'Category', 'Sentiment', 'Tweet']

# Перевірка на пропущені значення та заповнення їх порожнім рядком
data['Tweet'] = data['Tweet'].fillna('')

# Очищення тексту
data['cleaned_text'] = data['Tweet'].str.lower().str.replace(f'[{string.punctuation}]', '', regex=True)
data['cleaned_text'] = data['cleaned_text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Енкодинг меток
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])

# Розділення на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['Sentiment'], test_size=0.3, random_state=42)

# Токенізація та побудова словника
tokenizer = get_tokenizer('basic_english')

def yield_tokens(texts):
    for text in texts:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(X_train), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Токенізація та перетворення текстів у числові представлення
def text_to_sequence(texts):
    return [vocab(tokenizer(text)) for text in texts]

X_train_seq = text_to_sequence(X_train)
X_test_seq = text_to_sequence(X_test)

# Паддінг послідовностей
def pad_sequences(sequences, max_len):
    return [seq[:max_len] + [0] * (max_len - len(seq)) for seq in sequences]

max_len = 100
X_train_pad = pad_sequences(X_train_seq, max_len)
X_test_pad = pad_sequences(X_test_seq, max_len)

# Перетворення на PyTorch тензори
X_train_pad = torch.tensor(X_train_pad, dtype=torch.long)
X_test_pad = torch.tensor(X_test_pad, dtype=torch.long)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Створення PyTorch Dataset
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train_pad, y_train)
test_dataset = TextDataset(X_test_pad, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Визначення моделі
class RNNModelFromScratch(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RNNModelFromScratch, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)  # Ембедінги для слів
        output, hidden = self.rnn(embedded)  # RNN обчислення
        last_hidden = hidden[-1]  # Беремо останній стан прихованого шару
        return self.fc(last_hidden)  # Логіти

# Ініціалізація параметрів
vocab_size = 10000  # Приклад: розмір словника
embedding_dim = 128  # Розмірність ембедінгів
hidden_dim = 64  # Розмір прихованого шару
output_dim = 5  # Приклад: кількість класів (для класифікації)
num_epochs = 5  # Кількість епох
learning_rate = 0.001

# Створення моделі
model = RNNModelFromScratch(vocab_size, embedding_dim, hidden_dim, output_dim).to('cpu')

# Функція втрат і оптимізатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Припускаємо, що X_train і y_train вже підготовлені
# X_train має бути розмірності (batch_size, sequence_length)
# y_train має бути розмірності (batch_size)

# Генерація фейкових даних для прикладу
X_train = torch.randint(0, vocab_size, (100, 50)).to('cpu')  # 100 прикладів, послідовності довжиною 50
y_train = torch.randint(0, output_dim, (100,)).to('cpu')  # 100 міток

# Навчання
for epoch in range(num_epochs):
    model.train()  # Увімкнення режиму навчання
    optimizer.zero_grad()  # Скидання градієнтів

    # Передбачення
    predictions = model(X_train)
    loss = criterion(predictions, y_train)  # Розрахунок втрат

    loss.backward()  # Зворотне розповсюдження
    optimizer.step()  # Оновлення ваг

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Валідація
model.eval()  # Увімкнення режиму оцінки
with torch.no_grad():
    # Генерація фейкових тестових даних
    X_test = torch.randint(0, vocab_size, (20, 50)).to('cpu')  # 20 тестових прикладів
    y_test = torch.randint(0, output_dim, (20,)).to('cpu')  # 20 міток

    predictions = model(X_test)
    predicted_labels = torch.argmax(predictions, dim=1)  # Отримання класів
    accuracy = (predicted_labels == y_test).float().mean()  # Розрахунок точності

    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maksy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5, Loss: 1.6286876201629639
Epoch 2/5, Loss: 1.5749417543411255
Epoch 3/5, Loss: 1.5224155187606812
Epoch 4/5, Loss: 1.4710476398468018
Epoch 5/5, Loss: 1.4207502603530884
Test Accuracy: 30.00%


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random

# Function to load GloVe embeddings from a file
def load_glove_embeddings(glove_file, embedding_dim):
    glove_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_vectors[word] = vector
    return glove_vectors

# Load GloVe pre-trained embeddings (50-dimensional as an example)
glove_file = "glove.6B.50d.txt"  # Update with the correct path to the GloVe file
embedding_dim = 50

glove_vectors = load_glove_embeddings(glove_file, embedding_dim)

# Vocabulary (example vocabulary, replace with your actual vocab)
vocab = {"<PAD>": 0, "<UNK>": 1, "hello": 2, "world": 3}  # Example vocab
vocab_size = len(vocab)

# Initialize embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Build embedding matrix using pre-trained GloVe vectors
for word, idx in vocab.items():
    if word in glove_vectors:
        embedding_matrix[idx] = glove_vectors[word]
    else:
        # Random initialization for words not in the GloVe vocabulary
        embedding_matrix[idx] = np.random.normal(0, 1, embedding_dim)

# Define the RNN model with pre-trained embeddings
class RNNModelPretrained(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size, embedding_matrix):
        super(RNNModelPretrained, self).__init__()
        # Pre-trained embedding layer
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.embedding(x)
        h_0 = torch.zeros(1, x.size(0), hidden_dim).to(x.device)  # Initial hidden state
        out, _ = self.rnn(x, h_0)
        out = self.fc(out[:, -1, :])  # Use the output of the last RNN cell
        return out

# Initialize the model using pre-trained embeddings
hidden_dim = 128
output_size = 3  # Example number of classes
model_pretrained = RNNModelPretrained(vocab_size, embedding_dim, hidden_dim, output_size, embedding_matrix)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_pretrained = model_pretrained.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pretrained.parameters(), lr=0.001)

# Example training loop
num_epochs = 10
# Assume train_loader is a DataLoader with your dataset
for epoch in range(num_epochs):
    model_pretrained.train()
    total_loss = 0
    for batch_idx, (texts, labels) in enumerate(train_loader):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_pretrained(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}')

# Example evaluation loop
model_pretrained.eval()
correct = 0
total = 0
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model_pretrained(texts)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

# Classification report for detailed evaluation
from sklearn.metrics import classification_report

all_preds = []
all_labels = []
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model_pretrained(texts)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.50d.txt'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.metrics import classification_report

# Гіперпараметри
batch_size = 64
learning_rate = 0.001
epochs = 30

# Завантаження та трансформація датасету STL10
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Завантаження повного датасету
full_train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
full_test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Вибір підмножини з 1000 об'єктів
np.random.seed(42)  # Для відтворюваності
train_indices = np.random.choice(len(full_train_dataset), 800, replace=False)  # 800 об'єктів для тренування
test_indices = np.random.choice(len(full_test_dataset), 200, replace=False)  # 200 об'єктів для тестування

train_dataset = Subset(full_train_dataset, train_indices)
test_dataset = Subset(full_test_dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Огляд класів
classes = full_train_dataset.classes
print("Classes:", classes)

# Модифікація VGG-14
class ModifiedVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedVGG, self).__init__()
        # Завантаження попередньо навченого VGG-16
        vgg16 = models.vgg16(pretrained=True)
        
        # Замінюємо перший шар для роботи із зображеннями розміру 96x96
        vgg16.features[0] = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

        # Зберігаємо лише необхідні шари
        self.features = vgg16.features

        # Замінюємо класифікатор
        self.classifier = nn.Sequential(
            nn.Linear(512 * 3 * 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Ініціалізація моделі, оптимізатора і функції втрат
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModifiedVGG(num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Тренування
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Оцінка
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))


Files already downloaded and verified
Files already downloaded and verified
Classes: ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
Epoch 1/30, Loss: 2.3052, Accuracy: 4.69%
Epoch 1/30, Loss: 6.2823, Accuracy: 6.25%
Epoch 1/30, Loss: 8.6924, Accuracy: 7.29%
Epoch 1/30, Loss: 10.9913, Accuracy: 9.77%
Epoch 1/30, Loss: 13.2957, Accuracy: 9.69%
Epoch 1/30, Loss: 15.6014, Accuracy: 9.64%
Epoch 1/30, Loss: 17.8979, Accuracy: 10.27%
Epoch 1/30, Loss: 20.2322, Accuracy: 10.35%
Epoch 1/30, Loss: 22.5344, Accuracy: 9.90%
Epoch 1/30, Loss: 24.8415, Accuracy: 9.69%
Epoch 1/30, Loss: 27.1575, Accuracy: 9.66%
Epoch 1/30, Loss: 29.4621, Accuracy: 10.42%
Epoch 1/30, Loss: 31.7808, Accuracy: 10.12%
Epoch 2/30, Loss: 2.2996, Accuracy: 6.25%
Epoch 2/30, Loss: 4.6073, Accuracy: 6.25%
Epoch 2/30, Loss: 6.8898, Accuracy: 8.33%


KeyboardInterrupt: 